In [ ]:
# coding:utf-8
import argparse

import chainer
import chainer.links as L
from chainer.datasets import mnist, tuple_dataset
from chainer import iterators
from chainer import optimizers
from chainer import training
from chainer import serializers
from chainer.training import extensions
from chainer.dataset import convert
from chainer.functions.loss.mean_squared_error import mean_squared_error
#import cupy as cp
import numpy as np

from change_learning_rate import ChangeLearningRate
from stacked_denoising_autoencoder import StackedDenoisingAutoEncoder

import glob, os
from PIL import Image
import easydict

from matplotlib import pyplot as plt

"""
事前学習は2つのステップに分かれており、
　・1層毎にAutoEncoderとして学習を行う「Layer-Wise Pretrain」
　・ネットワーク全体を学習する「Fine Tuning」
"""

def load_image():
    x = 28
    y = x
    filepaths = glob.glob('test/*.png')
    f = open('text.txt', 'w') # 追記モードで開く    
    f.close()
    
    datasets = []
    i = 0
    
    for filepath in filepaths:
        basename = os.path.basename(filepath).split('.')[0]
        img = Image.open(filepath).convert('L')  #Pillowで読み込み。'L'はグレースケール
        #img = img.resize((x, y)) # 32x32xにリサイズ

        data = np.array(np.float32(img)/255.0)
        data = data.reshape(x * y) # 1行に変換
        t = np.array(i, dtype=np.int32) 
        
        datasets.append((data, t)) # データとラベルをタプルでリストに入れる

        f = open('text.txt', 'a') # 追記モードで開く
        f.write(str(i) + ', ' + basename + '\n') # 引数の文字列をファイルに書き込む
        f.close()
        
        i += 1
#     random.shuffle(datasets) # シャッフル
#     train = datasets[:1000] # 最初の千個を学習用
#     test = datasets[1000:1100] # 千個めから1100個目までをテスト用

    return datasets, None # 学習用、テスト用に分けず作成したデータセットごと返す

def pretraining():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--gpu', type=int, default=-1)
#     parser.add_argument('--seed', type=int, default=0)
#     parser.add_argument('--batchsize', type=int, default=256)
#     args = parser.parse_args()

    args = easydict.EasyDict({
        "gpu": -1,
        "seed": 0,
        "model_seed": 0,
        "batchsize": 8
    }) 

    xp = np
    gpu_id = args.gpu
    seed = args.seed
    train, _ =  load_image() # mnist.get_mnist()
    
    train, _ = convert.concat_examples(train, device=gpu_id)
    print('input' + str(train.shape[1]))
    batchsize = args.batchsize
    model = StackedDenoisingAutoEncoder(input_dim=train.shape[1])
    if chainer.cuda.available and args.gpu >= 0:
        xp = cp
        model.to_gpu(gpu_id)
    xp.random.seed(seed)
    print(type(train))
    print(train[1])

    # Layer-Wise Pretrain
    # 各層を取得しAutoEncoderとして学習する
    print("Layer-Wise Pretrain")
    for i, dae in enumerate(model.children()):
        print("Layer {}".format(i+1))
        train_tuple = tuple_dataset.TupleDataset(train, train)
        train_iter = iterators.SerialIterator(train_tuple, batchsize)
        clf = L.Classifier(dae, lossfun=mean_squared_error)
        clf.compute_accuracy = False
        if chainer.cuda.available and args.gpu >= 0:
            clf.to_gpu(gpu_id)
        optimizer = optimizers.MomentumSGD(lr=0.1)
        optimizer.setup(clf)
        updater = training.StandardUpdater(train_iter, optimizer, device=gpu_id)
        trainer = training.Trainer(updater, (50000, "iteration"), out="mnist_result")
        trainer.extend(extensions.LogReport())
        trainer.extend(extensions.PrintReport(['iteration', 'main/loss', 'elapsed_time']))
        # 20000iteration毎に学習率(lr)を1/10にする
        trainer.extend(ChangeLearningRate(), trigger=(20000, "iteration"))
        trainer.run()
        # 層毎の学習に合わせて学習データの次元を784->500->500->2000と変換していく
        train = dae.encode(train).data

    # Finetuning
    print("fine tuning")
    # Fine Tuning時はDropoutなし
    with chainer.using_config("train", False):
        train, _ = mnist.get_mnist()
        train, _ = convert.concat_examples(train, device=gpu_id)
        train_tuple = tuple_dataset.TupleDataset(train, train)
        train_iter = iterators.SerialIterator(train_tuple, batchsize)
        model = L.Classifier(model, lossfun=mean_squared_error)
        model.compute_accuracy = False
        if chainer.cuda.available and args.gpu >= 0:
            model.to_gpu(gpu_id)
        optimizer = optimizers.MomentumSGD(lr=0.1)
        optimizer.setup(model)
        updater = training.StandardUpdater(train_iter, optimizer, device=gpu_id)
        trainer = training.Trainer(updater, (100000, "iteration"), out="mnist_result")
        trainer.extend(extensions.LogReport())
        trainer.extend(extensions.PrintReport(['iteration', 'main/loss', 'elapsed_time']))
        trainer.extend(ChangeLearningRate(), trigger=(20000, "iteration"))
        trainer.run()

    outfile = "StackedDenoisingAutoEncoder-seed{}.model".format(seed)
    serializers.save_npz(outfile, model.predictor)


if __name__ == '__main__':
    pretraining()

input784
<class 'numpy.ndarray'>
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.   

iteration   main/loss   elapsed_time
13          0.100006    0.874602      
25          0.095632    2.12349       
37          0.0851992   3.7697        
49          0.069429    4.71991       
62          0.0600351   6.60596       
74          0.0593757   7.22251       
86          0.0589908   8.04739       
98          0.0573731   8.78076       
111         0.0566976   9.47816       
123         0.0563162   10.3285       
135         0.0554592   11.1283       
147         0.0537512   11.8538       
160         0.0522248   12.7939       
172         0.051917    13.6963       
184         0.0499315   15.0347       
196         0.0488576   16.4726       
209         0.0467375   17.3196       
221         0.0464233   18.5074       
233         0.0449742   19.4561       
245         0.0436392   20.5807       
258         0.0432129   21.6534       
270         0.0412647   22.3731       
282         0.0412499   23.1145       
294         0.0405571   24.1453       
307         0.0396803   24.

2585        0.0120418   253.724       
2597        0.0117554   254.764       
2610        0.0117271   255.891       
2622        0.011747    257.052       
2634        0.0116618   258.209       
2646        0.0119844   259.343       
2659        0.0116453   260.646       
2671        0.0114728   261.474       
2683        0.011409    262.284       
2695        0.0114749   263.009       
2708        0.0115694   263.743       
2720        0.0118195   264.457       
2732        0.0116942   265.144       
2744        0.0114221   265.838       
2757        0.0116122   266.613       
2769        0.0114431   267.379       
2781        0.0114196   268.087       
2793        0.0112426   268.802       
2806        0.011181    269.686       
2818        0.0112829   270.761       
2830        0.010895    272.116       
2842        0.0112097   273.627       
2855        0.0113251   275.821       
2867        0.0109756   277.247       
2879        0.0110692   278.631       
2891        0.0110922   2

5170        0.00755803  473.884       
5182        0.00769503  475.337       
5194        0.00779944  477.425       
5207        0.00752069  478.952       
5219        0.00778165  480.561       
5231        0.00772543  482.037       
5243        0.00760376  484.694       
5256        0.00756777  487.342       
5268        0.00769016  488.924       
5280        0.00759321  489.871       
5292        0.00770871  491.861       
5305        0.00737412  493.484       
5317        0.00758107  497.123       
5329        0.00757588  498.986       
5341        0.00762888  500.988       
5354        0.00748852  502.411       
5366        0.00732151  504.873       
5378        0.00750416  505.816       
5390        0.00741478  507.057       
5403        0.00751778  508.383       
5415        0.00766488  509.294       
5427        0.00734026  511.987       
5439        0.0075408   517.26        


In [ ]:
rows = 256
cols = rows

img = Image.open('whale/images.jpg').convert('L')  #Pillowで読み込み。'L'はグレースケールを意味する
img = img.resize((rows, cols)) # 32x32xにリサイズ
label = 0 # ラベル(ファイル名)

x = np.array(img, dtype=np.float32)
x = x.reshape(rows*cols)

print(x)
# x = x.reshape(1, 32, 32) # (チャネル、高さ、横幅)
# t = np.array(label, dtype=np.int32) 
hoge = train[0][0].reshape(28, 28)
plt.imshow(train[0][0].reshape(28, 28), cmap='gray')
plt.imshow(x.reshape(rows, cols),cmap='gray')
plt.show()